# Census API Example B
A variation of example A, this example includes an API key, dumping retrieved data to an output file, and subsequently working with data loaded from output. Without a key you are limited to 500 requests per day. By storing data after retrieval and then loading it from storage, you can make modifications to your script without having to run the api block again (assuming that you retrieved the data that you need correctly).

Example uses Population Estimates 2018 Vintage:
https://www.census.gov/data/developers/data-sets/popest-popproj/popest.Vintage_2018.html

Find ANSI / FIPS codes for geographies:
* Census Bureau: https://www.census.gov/library/reference/code-lists/ansi.html
* MCDC: https://census.missouri.edu/geocodes/

## Set Variables and Retrieve Data

In [ ]:
%matplotlib inline
import requests, json, pandas as pd

In [ ]:
#Set variables
year='2018'
dsource='pep'
dname='components'
cols='GEONAME,NATURALINC,DOMESTICMIG,INTERNATIONALMIG'
state='36'
county='005,047,061,081,085'
keyfile='census_key.txt'
outfile='zbp2017.json'

In [ ]:
base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
base_url

In [ ]:
#Read api key in from file
with open(keyfile) as key:
    api_key=key.read().strip()

In [ ]:
#Retrieve data, write output when finished
data_url = f'{base_url}?get={cols}&for=county:{county}&in=state:{state}&key={api_key}'
response=requests.get(data_url)
outdata=response.json()
with open(outfile, 'w') as f:
    json.dump(outdata, f)
print('Data dumped to json file')
print('File has', len(outdata), 'records')

## Pandas Examples

In [ ]:
#Load data from output file, convert to dataframe, perform some basic operations
with open(outfile, 'r') as f:
    data=json.load(f)
df=pd.DataFrame(data[1:], columns=data[0]).\
rename(columns={"NATURALINC": "Natural Increase", "DOMESTICMIG": "Net Domestic Mig", "INTERNATIONALMIG":"Net Foreign Mig"})
df['fips']=df.state+df.county
df.set_index('fips',inplace=True)
df.drop(columns=['state','county'],inplace=True)
df=df.astype(dtype={'Natural Increase':'int64','Net Domestic Mig':'int64','Net Foreign Mig':'int64'},inplace=True)
df

In [ ]:
#Total population change 2017-2018
df.sum(axis=1)

In [ ]:
#Create a basic chart
labels=df['GEONAME'].str.split(',',expand=True)[0].str.replace('County','')
ax=df.plot.bar(rot=0, title='Components of Population Change 2017-18')
ax.set_xticklabels(labels)
ax.set_xlabel('')

## Obtain List of Variables

In [ ]:
vars_url=f'https://api.census.gov/data/{year}/{dsource}/{dname}/variables.json'
response=requests.get(vars_url)

#"variables" is top key in the file - reference it to flatten the file so individual variables become keys 
variables=response.json()['variables']
variables

In [ ]:
#Iterate through keys and get specific sub-values
for k,v in variables.items():
    print(k,v['label'])

In [ ]:
#Iterate even further to capture specific code lists
regions=variables['DIVISION']['values']['item']
regions